In [1]:
import os
import sys
sys.path.append('../')
from lib.env.global_var import *
from lib.func.shelve_store import *
sys.path.remove('../')

In [2]:
%run -i test.py

CPIdefined
GDPdefined
DGS10defined
HPIdefined
PAYEMSdefined
TEDRATEdefined
FEDFUNDSdefined
NETEXPdefined
PCEdefined
UNRATEdefined
CSENTdefined
OASdefined
RECESSIONdefined
VIXCLSdefined


In [3]:
from sklearn import metrics

def mean_squared_log_error(y_true,y_pred):
    diff = (y_true+1).apply(np.log).values - (y_pred+1).apply(np.log).values
    return np.sqrt(np.mean(diff**2))

def accuracy_score_function(y_true, y_pred):
    temp = pd.concat([y_true, y_pred],1).dropna()
    score_df = pd.DataFrame(map(lambda x: x[0]==x[1],temp.values))*1
    score = np.mean(score_df)
    return(score)

def evaluate_metric(y_true,y_pred,function='mse'):
    if function == 'evs':
        metric_value = metrics.explained_variance_score(y_true, y_pred)
    elif function == 'mae':
        metric_value = metrics.mean_absolute_error(y_true, y_pred)
    elif function == 'mse':
        metric_value = metrics.mean_squared_error(y_true, y_pred)
    elif function == 'msle':
        metric_value = mean_squared_log_error(y_true, y_pred)
    elif function == 'mdae':
        metric_value = metrics.median_absolute_error(y_true, y_pred)
    elif function == 'r2':
        metric_value = metrics.r2_score(y_true, y_pred)
    elif function =='acf':
        metric_value = accuracy_score_function(y_true, y_pred)
    return metric_value

In [4]:
from sklearn.model_selection import KFold

In [5]:
def data_prep(sector_dir_str,sector_sheet_int,spx_dir_str):
    sector_ind = read_data(sector_dir_str,sector_sheet_int)
    sector_ind_ret = sector_ind.shift()/sector_ind - 1
    
    spx = read_csv(spx_dir_str).resample("M").last()
    spx_ret = spx.shift()/spx -1

    sector_ind_ret,spx_ret = model_data(sector_ind_ret,spx_ret)
    defeat_benchmark = pd.DataFrame((sector_ind_ret.values>spx_ret.values),
                                    index= sector_ind_ret.index,columns=['beat_benchmark'])*1
    x = pd.concat(data_set_selection,1).dropna()
    x,y = model_data(x,defeat_benchmark)
    return(x,y)

In [6]:
x,y = data_prep(benchmark_str,7,spx_str)

In [7]:
# # for one sector perform cross validation
# # 4: IYJ US Equity

# sheet_int = 7
# sector_ind = read_data(benchmark_str,sheet_int)
# sector_ind.dropna().head()

In [8]:
# sector_ind_ret = sector_ind.shift()/sector_ind - 1

# spx = read_csv(spx_str).resample("M").last()
# spx_ret = spx.shift()/spx -1

# sector_ind_ret,spx_ret = model_data(sector_ind_ret,spx_ret)
# defeat_benchmark = pd.DataFrame((sector_ind_ret.values>spx_ret.values),
#                                 index= sector_ind_ret.index,columns=['beat_benchmark'])*1
# spx.head()

In [9]:
# defeat_benchmark.head()

In [10]:
# x = pd.concat(data_set_selection,1).dropna()
# x,y = model_data(x,defeat_benchmark)

In [11]:
# y.index

In [12]:
def K_Fold_evaluation(x,y,n,penalty,C,evaluation_metric='mse'):
    kf = KFold(n_splits=n)
    kf.get_n_splits(x)

    # x : train_dict[key][0]
    # y : train_dict[key][1]
    train_dict ={}
    test_dict = {}
    ind = 1
    for train_index, test_index in kf.split(x):
        x_train, x_test = x.iloc[train_index,:],x.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index,:],y.iloc[test_index,:]

        train_dict[ind] = [x_train,y_train]
        test_dict[ind] = [x_test,y_test]
        ind += 1
        
    metric_list =[]
    for select_index in range(1,n+1):
        x_train_model = train_dict[select_index][0]
        x_test_model = test_dict[select_index][0]

        y_train_model = train_dict[select_index][1]
        y_test_model = test_dict[select_index][1]


        logistic  = LogisticRegression(penalty=penalty, C = C)
        logistic.fit(x_train_model,y_train_model)

        y_predict = pd.DataFrame(logistic.predict(x_test_model),index = x_test_model.index)
        metric_value = evaluate_metric(y_test_model,y_predict,evaluation_metric)
        metric_list.append(metric_value)
    
    # the higher the better
    overall_metric = np.mean(metric_list)
#     np.prod(map(lambda x: x,metric_list))
    out_df = pd.DataFrame([penalty,1/C,overall_metric],index = ['penalty','lambda','overall_metric']).T
    return(out_df)

In [13]:
n=3

index = 1
penalty_list = ['l1','l2']
C_list = [1e-5,1e-4,1e-3,1e-2,1e-1,1,10.0,100.0,1000.0]
overall_df = pd.DataFrame(index = map(lambda x:1/x,C_list),columns=penalty_list)
overall_df.index.name = 'lambda'
overall_df.columns.name = 'penalty'
for penalty in penalty_list:
    for C in C_list:
        result_df = K_Fold_evaluation(x,y,n,penalty,C,evaluation_metric='acf')
        result_df.index = [index]
        index += 1
        overall_df.loc[1/C,penalty]=result_df.loc[:,'overall_metric'].values[0]
overall_df

/home/luke/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


penalty,l1,l2
lambda,,
100000.000,0.515152,0.5
10000.000,0.515152,0.545455
1000.000,0.515152,0.530303
100.000,0.515152,0.5
10.000,0.530303,0.530303
1.000,0.409091,0.5
0.100,0.469697,0.515152
0.010,0.469697,0.5
0.001,0.469697,0.5


[1000.0, 100.0, 10.0]

In [9]:
n = 3

kf = KFold(n_splits=n)
kf.get_n_splits(x)

# x : train_dict[key][0]
# y : train_dict[key][1]
train_dict ={}
test_dict = {}
ind = 1
for train_index, test_index in kf.split(x):
    x_train, x_test = x.iloc[train_index,:],x.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index,:],y.iloc[test_index,:]
    
    train_dict[ind] = [x_train,y_train]
    test_dict[ind] = [x_test,y_test]
    ind += 1

In [15]:
penalty = 'l2'
C = 1e-4

metric_list =[]
for select_index in range(1,n+1):
    x_train_model = train_dict[select_index][0]
    x_test_model = test_dict[select_index][0]

    y_train_model = train_dict[select_index][1]
    y_test_model = test_dict[select_index][1]


    logistic  = LogisticRegression(penalty=penalty, C = C)
    logistic.fit(x_train_model,y_train_model)

    y_predict = pd.DataFrame(logistic.predict(x_test_model),index = x_test_model.index)
    metric_value = evaluate_metric(y_test_model,y_predict,'msle')
    metric_list.append(metric_value)

overall_metric = np.prod(map(lambda x: 1/x,metric_list))

,penalty,lambda,overall_metric
0,l2,10000,6.0952


0.54185807659592389

In [3]:
result_dict={}
model_dict={}

In [4]:
# Edit: input sector_ind excel sheet instead
# Put data_set_selection into input variable
for sheet_int in np.arange(1,10):
    sector_ind = read_data(benchmark_str,sheet_int)
    name,score,model = logistic_reg_sector_index(data_set_selection, sector_ind ,
                        base_benchmark=spx_ret, print_plot = False, penalty = 'l2', C = 1e-4)
    result_dict[name] = score
    model_dict[name] = model

Sector selected: 
IYZ US Equity
Coeff
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0        -0.003805 -0.000002   -0.000036  0.000024 -0.000838  0.000881   

        PCE    UNRATE  RECPROUSM156N  VIXCLS  
0  0.000004 -0.000561       0.000769  0.0018  
prediction rate: 32.143%



Sector selected: 
IYK US Equity


/home/luke/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Coeff
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0          0.00448 -0.000004   -0.000004 -0.000232  0.000706 -0.000891   

        PCE    UNRATE  RECPROUSM156N    VIXCLS  
0  0.000002 -0.000294      -0.000729 -0.001479  
prediction rate: 50.0%



Sector selected: 
IYE US Equity
Coeff
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0         0.004081 -0.000012    0.000045  0.000127  0.000946 -0.000856   

        PCE    UNRATE  RECPROUSM156N    VIXCLS  
0 -0.000004  0.000791       0.000776  0.002121  
prediction rate: 39.286%



Sector selected: 
IYJ US Equity
Coeff
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0         0.004488 -0.000002    0.000084  0.000059 -0.001257 -0.000876   

        PCE    UNRATE  RECPROUSM156N    VIXCLS  
0  0.000002 -0.000312      -0.000876 -0.001388  
prediction rate: 57.143%



Sector selected: 
IYM US Equity
Coeff
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDR

ValueError: Found array with 0 sample(s) (shape=(0, 10)) while a minimum of 1 is required.

In [5]:
result_dict.items()

[('IYZ US Equity', 0.32142857142857145),
 ('IYK US Equity', 0.5),
 ('IDU US Equity', 0.4642857142857143),
 ('IYF US Equity', 0.32142857142857145),
 ('IYW US Equity', 0.7857142857142857),
 ('IYJ US Equity', 0.5714285714285714),
 ('IYM US Equity', 0.6071428571428571),
 ('ISHAF US Equity', 0.73913043478260865),
 ('IYE US Equity', 0.39285714285714285)]

In [6]:
pd.concat(data_set_selection, 1).columns

Index([u'CPALTT01USQ657N', u'GDP', u'CSUSHPINSA', u'DGS10', u'TEDRATE',
       u'FEDFUNDS', u'PCE', u'UNRATE', u'RECPROUSM156N', u'VIXCLS'],
      dtype='object')

In [7]:
col_name = pd.concat(data_set_selection, 1).columns
for key in model_dict.keys():
    print key
    model = model_dict[key]
    coef_df = pd.DataFrame(model.coef_, columns = col_name)
    print coef_df

IYZ US Equity
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0        -0.003805 -0.000002   -0.000036  0.000024 -0.000838  0.000881   

        PCE    UNRATE  RECPROUSM156N  VIXCLS  
0  0.000004 -0.000561       0.000769  0.0018  
IYK US Equity
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0          0.00448 -0.000004   -0.000004 -0.000232  0.000706 -0.000891   

        PCE    UNRATE  RECPROUSM156N    VIXCLS  
0  0.000002 -0.000294      -0.000729 -0.001479  
IDU US Equity
   CPALTT01USQ657N       GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0         0.003657  0.000007   -0.000069 -0.000087 -0.000948 -0.000979   

        PCE    UNRATE  RECPROUSM156N    VIXCLS  
0  0.000003  0.000717       0.000788  0.001973  
IYF US Equity
   CPALTT01USQ657N           GDP  CSUSHPINSA     DGS10   TEDRATE  FEDFUNDS  \
0         0.003999  9.459046e-07   -0.000018  0.000083  0.000768  0.000945   

            PCE    UNRATE  RECPROUSM156N    VIXCLS